In [2]:
# importing necessities
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

In [3]:
# loading API / key
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# setting up path variables
DATA_DIR = "/Users/nataliepegues/data4380.np/llm_workspace/data"
SUMMARIES_JSON = f"{DATA_DIR}/summaries.json"
COMPARISONS_JSON = f"{DATA_DIR}/comparisons.json"

with open(SUMMARIES_JSON, "r") as f:
    summaries = json.load(f)

# function to compare abstracts / summaries
def compare_abstract_summary(abstract, summary):
    prompt = f"""
You are an expert in psychology research. Compare the following abstract and summary:

Abstract:
{abstract}

Summary:
{summary}

Please:
- Rate how well the summary captures the abstract's key points on a scale from 1 (poor) to 5 (excellent).
- Provide a brief explanation of your rating.

Respond ONLY in JSON format like:
{{"rating": <number>, "explanation": "<text>"}}
"""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo", # model choice, using same as summarization
        messages=[{"role": "user", "content": prompt}],
        max_tokens=200,
        temperature=0,
    )
    return response.choices[0].message.content

comparisons = []

for entry in summaries:
    filename = entry["filename"]
    abstract = entry.get("abstract", "")
    summary = entry.get("summary", "")

    # adding text as a 'pulse check'
    print(f"Comparing summary to abstract for {filename}...")
    result_text = compare_abstract_summary(abstract, summary)

    # trying to safely parse JSON
    try:
        import json
        result_json = json.loads(result_text)
    except json.JSONDecodeError:
        print(f"Warning: Failed to parse JSON response for {filename}. Raw response:")
        print(result_text)
        result_json = {"rating": None, "explanation": result_text}

    comparisons.append({
        "filename": filename,
        "rating": result_json.get("rating"),
        "explanation": result_json.get("explanation"),
    })

# saving comparison
with open(COMPARISONS_JSON, "w") as f:
    json.dump(comparisons, f, indent=2)

# completion message
print(f"Comparison results saved to {COMPARISONS_JSON}")


Comparing summary to abstract for psychpaper14.pdf...
Comparing summary to abstract for psychpaper15.pdf...
Comparing summary to abstract for psychpaper01.pdf...
Comparing summary to abstract for psychpaper17.pdf...
Comparing summary to abstract for psychpaper03.pdf...
Comparing summary to abstract for psychpaper02.pdf...
Comparing summary to abstract for psychpaper16.pdf...
Comparing summary to abstract for psychpaper12.pdf...
Comparing summary to abstract for psychpaper07.pdf...
Comparing summary to abstract for psychpaper13.pdf...
Comparing summary to abstract for psychpaper05.pdf...
Comparing summary to abstract for psychpaper11.pdf...
Comparing summary to abstract for psychpaper10.pdf...
Comparing summary to abstract for psychpaper04.pdf...
Comparing summary to abstract for psychpaper21.pdf...
Comparing summary to abstract for psychpaper09.pdf...
Comparing summary to abstract for psychpaper08.pdf...
Comparing summary to abstract for psychpaper20.pdf...
Comparing summary to abstrac